Load the training data

In [1]:
from google.colab import files
import os

filename = "housing_data_simulation.csv"

# Check if the file already exists
if not os.path.exists(filename):
    # If it doesn't exist, upload it
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]  # Get the filename of the uploaded file
    print(f"File '{filename}' uploaded successfully.")
else:
    print(f"File '{filename}' already exists. Skipping upload.")

Saving housing_data_simulation.csv to housing_data_simulation.csv
File 'housing_data_simulation.csv' uploaded successfully.


If necessary, install bitsandbytes

In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

Train the model

In [3]:
import os
from huggingface_hub import login
from google.colab import userdata


# Your current token setup code is fine, just make sure it runs before your model code
hf_token = userdata.get('llama3.1token')  # or use os.environ.get('HUGGINGFACE_TOKEN')
os.environ['HUGGINGFACE_TOKEN'] = hf_token
login(token=hf_token)

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    LlamaTokenizer,
    LlamaForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from sklearn.model_selection import train_test_split
import random
from tqdm import tqdm
import os
from peft import get_peft_model, LoraConfig, TaskType
import bitsandbytes


# Set random seeds for reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_val)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Load and preprocess the data
def load_data(file_path):
    """Load and preprocess the dataset."""
    df = pd.read_csv(file_path)

    # Convert categorical features to strings
    df['Bedrooms'] = df['Bedrooms'].astype(str)
    df['Bathrooms'] = df['Bathrooms'].astype(str)

    # Normalize numerical features
    df['Price'] = df['Price'] / 1000000  # Convert to millions
    df['Square Footage'] = df['Square Footage'] / 1000  # Convert to thousands
    df['Lot Size (Acres)'] = df['Lot Size (Acres)']  # Keep as is

    # Process features list into a string
    if isinstance(df['Features'].iloc[0], str):
        # If features are stored as strings that look like lists, convert them
        try:
            df['Features'] = df['Features'].apply(eval).apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        except:
            # If eval fails, assume it's already a string
            pass
    elif isinstance(df['Features'].iloc[0], list):
        df['Features'] = df['Features'].apply(lambda x: ', '.join(x))

    return df

# Custom dataset class for Llama
class RealEstateLlamaDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Create input text with a clear instruction for the model
        property_text = f"Price: ${row['Price']:.2f}M, Bedrooms: {row['Bedrooms']}, Bathrooms: {row['Bathrooms']}, " \
                         f"Square Footage: {row['Square Footage']:.1f}K sq ft, Lot Size: {row['Lot Size (Acres)']:.2f} acres, " \
                         f"Features: {row['Features']}"

        # Create full prompt in the instruction format Llama understands
        full_text = f"<s>[INST] Write a compelling and accurate real estate listing description for this property: \n\n{property_text} [/INST] {row['Listing Description']}</s>"

        # Tokenize full text
        encodings = self.tokenizer(
            full_text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )

        # Create labels - set to -100 for the instruction part so it's not included in loss calculation
        labels = encodings['input_ids'].clone()

        # Find where the [/INST] token ends
        inst_tokens = self.tokenizer.encode(" [/INST]", add_special_tokens=False)
        inst_end_positions = []

        for i in range(len(labels[0]) - len(inst_tokens) + 1):
            if labels[0][i:i+len(inst_tokens)].tolist() == inst_tokens:
                inst_end_positions.append(i + len(inst_tokens) - 1)

        if inst_end_positions:
            # Set all tokens before the end of [/INST] to -100
            labels[0, :inst_end_positions[0]+1] = -100

        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'labels': labels.squeeze(),
            'raw_property': property_text,
            'raw_description': row['Listing Description']
        }

# Training function using Trainer API
def train_llama_model(model, train_dataset, val_dataset, tokenizer, output_dir, epochs=1, lr=2e-4, batch_size=1):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch
        warmup_steps=50,  # Reduce warmup steps
        weight_decay=0.01,
        logging_dir=f"{output_dir}/logs",
        logging_steps=10,
        eval_strategy="steps",  # Use eval_strategy instead of evaluation_strategy
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=2,  # Keep only 2 checkpoints to save space
        load_best_model_at_end=True,
        report_to="none",
        learning_rate=lr,
        fp16=True,  # Use fp16 for training
        optim="paged_adamw_8bit",  # Memory-efficient optimizer
        max_grad_norm=0.3,  # Gradient clipping
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
    )

    print("Starting training...")
    trainer.train()

    print("Training complete. Saving model...")
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    return model

# Generate a sample description
def generate_sample(model, tokenizer, dataloader):
    model.eval()

    # Get a random sample
    batch = next(iter(dataloader))
    property_info = batch['raw_property'][0]
    actual_description = batch['raw_description'][0]

    print("\nSample Property:")
    print(property_info)

    print("\nActual Description:")
    print(actual_description)

    # Generate description
    generated_text = generate_listing_description(model, tokenizer, property_info)

    print("\nGenerated Description:")
    print(generated_text)

# Generate listing description
def generate_listing_description(model, tokenizer, property_info, max_length=150):
    model.eval()

    # Prepare input text with prompt in the instruction format
    input_text = f"<s>[INST] Write a compelling and accurate real estate listing description for this property. Make sure that the adjectives you use to describe the house are appropriate considering the features of the house such as Price, Square footage and lot size. For example, you would not want to use the word expansive to describe a 0.1 acre lot or the word cozy to describe a 5000 square foot home. Any mention of Price, Square Footage and/or Lot Size in the listing description should EXACTLY match these (input) numeric values without rounding or changing them: Price: ${property_info.split('Price: $')[1].split('M')[0]}M, Square Footage: {property_info.split('Square Footage: ')[1].split('K')[0]}K sq ft, Lot Size: {property_info.split('Lot Size: ')[1].split(' acres')[0]} acres. \n\n{property_info} [/INST]"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=False)

    # Extract just the generated description by finding everything after [/INST]
    description_part = generated_text.split("[/INST]")[1].split("</s>")[0].strip()

    return description_part

# Function to format a new property for prediction
def format_property_input(property_dict):
    price_in_millions = property_dict['Price'] / 1000000
    sq_ft_in_thousands = property_dict['Square Footage'] / 1000

    features = property_dict['Features']
    if isinstance(features, list):
        features = ', '.join(features)

    property_text = f"Price: ${price_in_millions:.2f}M, Bedrooms: {property_dict['Bedrooms']}, " \
                     f"Bathrooms: {property_dict['Bathrooms']}, " \
                     f"Square Footage: {sq_ft_in_thousands:.1f}K sq ft, " \
                     f"Lot Size: {property_dict['Lot Size (Acres)']:.2f} acres, " \
                     f"Features: {features}"

    return property_text

# Setup LoRA for parameter-efficient fine-tuning
def setup_lora_model(base_model_path="meta-llama/Llama-3.1-8B", hf_token=None):
    print(f"Loading base model: {base_model_path}...")

    from transformers import AutoTokenizer, AutoModelForCausalLM

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_path,
        token=hf_token,
        use_fast=True
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token



    # Configure 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    # Load the model with 4-bit quantization
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=bnb_config,
        device_map="auto",
        token=hf_token
    )

    # Define LoRA configuration
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,  # Lower rank for less memory
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"]  # Optimize for memory: target fewer modules
    )

    # Get the PEFT model
    model = get_peft_model(model, peft_config)

    # Print trainable parameters
    print("Trainable parameters:")
    model.print_trainable_parameters()

    return model, tokenizer

# Main execution
def main(file_path, output_dir="./llama3_real_estate_model", batch_size=4, epochs=3, model_path="meta-llama/Llama-3.1-8B", hf_token=None):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load data
    df = load_data(file_path)

    # Split the data
    train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

    # Initialize model and tokenizer with LoRA for efficient fine-tuning
    model, tokenizer = setup_lora_model(model_path, hf_token)

    # Create datasets
    train_dataset = RealEstateLlamaDataset(train_df, tokenizer)
    val_dataset = RealEstateLlamaDataset(val_df, tokenizer)

    # Train model
    trained_model = train_llama_model(
        model,
        train_dataset,
        val_dataset,
        tokenizer,
        output_dir,
        epochs=epochs,
        batch_size=batch_size
    )

    # Create a small dataloader for generating samples
    sample_dataloader = DataLoader(val_dataset, batch_size=1)

    # Generate a sample
    generate_sample(trained_model, tokenizer, sample_dataloader)

    # Test with a new property
    test_property = {
        'Price': 450000,
        'Bedrooms': '3',
        'Bathrooms': '2',
        'Square Footage': 1800,
        'Lot Size (Acres)': 0.25,
        'Features': ['Hardwood floors', 'Updated kitchen', 'Finished basement', 'Deck']
    }

    property_text = format_property_input(test_property)
    description = generate_listing_description(trained_model, tokenizer, property_text)

    print("\nGenerated listing for test property:")
    print(description)


#hf_token = os.environ.get('HUGGINGFACE_TOKEN', 'COPY TOKEN HERE')

main(
    file_path=filename,
    output_dir="./llama3_real_estate_model",
    batch_size=1,
    epochs=2,
    model_path="meta-llama/Llama-3.1-8B",
    hf_token=hf_token
)


Using cuda device
Loading base model: meta-llama/Llama-3.1-8B...


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss
100,0.141100,0.136263
200,0.073800,0.071647
300,0.064400,0.064445
400,0.061900,0.062378


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Training complete. Saving model...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Sample Property:
Price: $0.37M, Bedrooms: 4, Bathrooms: 2.5, Square Footage: 1.8K sq ft, Lot Size: 0.47 acres, Features: outdoor living spaces, custom lighting, designer windows, spa-like bathrooms, high-end appliances, smart home features, walk-in closets, gourmet kitchen, hardwood floors

Actual Description:
Welcome to your dream home! This breathtaking 4-bedroom residence redefines modern living. With 2.5 meticulously designed bathrooms and 1774 square feet of living space, every inch has been carefully considered. The 0.5-acre lot provides ample room for your lifestyle. Highlights include custom lighting, hardwood floors, walk-in closets. At $368,600.0, this home is a rare find you won't want to miss.

Generated Description:
Experience luxury reimagined in this remarkable 4-bedroom marvel. Situated in a historic district, the 1757 square foot floor plan maximizes both style and function. The 0.5-acre lot provides ample room for your lifestyle. Featuring elements like walk-in close

In [6]:
from google.colab import files
import os

filename = "housing_data_simulation test copy.csv"

# Check if the file already exists
if not os.path.exists(filename):
    # If it doesn't exist, upload it
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]  # Get the filename of the uploaded file
    print(f"File '{filename}' uploaded successfully.")
else:
    print(f"File '{filename}' already exists. Skipping upload.")

File 'housing_data_simulation test copy.csv' already exists. Skipping upload.


Test the model on 50 observations for later annotation

In [5]:
import re

def post_process_description(generated_text, property_info):
    """
    Ensures that numerical values in the generated text match the desired format:
    - Price: $300,000 (full dollar amount with commas)
    - Square footage: 1234 (whole number)
    - Acres: 0.15 (decimal number)

    Args:
        generated_text (str): The generated listing description
        property_info (str): The original property information string

    Returns:
        str: The processed description with correctly formatted numerical values
    """
    # Extract the original values from property_info
    price_match = re.search(r'Price: \$([\d.]+)M', property_info)
    sqft_match = re.search(r'Square Footage: ([\d.]+)K', property_info)
    lot_match = re.search(r'Lot Size: ([\d.]+) acres', property_info)

    # Convert to the desired formats
    if price_match:
        # Convert $X.YM to $X,YY0,000
        price_in_millions = float(price_match.group(1))
        price_in_dollars = int(price_in_millions * 1000000)
        formatted_price = "${:,}".format(price_in_dollars)
    else:
        formatted_price = None

    if sqft_match:
        # Convert X.YK to whole number
        sqft_in_thousands = float(sqft_match.group(1))
        formatted_sqft = str(int(sqft_in_thousands * 1000))
    else:
        formatted_sqft = None

    if lot_match:
        # Keep exact decimal format for acreage
        formatted_lot = lot_match.group(1)
    else:
        formatted_lot = None

    # Only proceed with replacements if we have valid values
    if not (formatted_price or formatted_sqft or formatted_lot):
        print("Warning: Could not extract values from property_info")
        return generated_text

    processed_text = generated_text

    # Replace price variations
    if formatted_price:
        # Handle $X.YM format
        processed_text = re.sub(r'\$([\d,.]+)M', formatted_price, processed_text)
        processed_text = re.sub(r'\$([\d,.]+) million', formatted_price, processed_text)
        processed_text = re.sub(r'\$([\d,.]+) Million', formatted_price, processed_text)

        # Handle normal dollar amounts
        processed_text = re.sub(r'\$([\d,]+)', formatted_price, processed_text)

        # Handle spelled out numbers
        million_words = ['million', 'Million']
        for word in million_words:
            pattern = rf'(\$[\d,.]+ {word}|\$[\d,.]+{word})'
            processed_text = re.sub(pattern, formatted_price, processed_text)

    # Replace square footage variations
    if formatted_sqft:
        # Handle XK sq ft format
        processed_text = re.sub(r'([\d,.]+)K sq ft', formatted_sqft + " sq ft", processed_text)
        processed_text = re.sub(r'([\d,.]+)K square feet', formatted_sqft + " square feet", processed_text)

        # Handle normal square footage formats
        processed_text = re.sub(r'([\d,.]+) sq\. ft\.', formatted_sqft + " sq. ft.", processed_text)
        processed_text = re.sub(r'([\d,.]+) square feet', formatted_sqft + " square feet", processed_text)
        processed_text = re.sub(r'([\d,.]+) sf', formatted_sqft + " sq ft", processed_text)

        # Handle thousand variations
        thousand_words = ['thousand square feet', 'thousand sq ft', 'thousand sq. ft.']
        for word in thousand_words:
            pattern = rf'([\d,.]+) {word}'
            processed_text = re.sub(pattern, formatted_sqft + " square feet", processed_text)

    # Replace lot size variations
    if formatted_lot:
        # Standard lot size formats
        processed_text = re.sub(r'([\d,.]+) acre(?!s)', formatted_lot + " acre", processed_text)
        processed_text = re.sub(r'([\d,.]+) acres', formatted_lot + " acres", processed_text)
        processed_text = re.sub(r'([\d,.]+)-acre', formatted_lot + "-acre", processed_text)

    # Check if values were properly inserted and add them if missing
    if formatted_price and formatted_price not in processed_text:
        processed_text = f"Priced at {formatted_price}. " + processed_text

    if formatted_sqft and formatted_sqft + " sq" not in processed_text:
        processed_text = f"This {formatted_sqft} sq ft home " + processed_text

    if formatted_lot and formatted_lot + " acre" not in processed_text:
        if "acre" in processed_text:
            # Try to insert near an existing mention of acres
            processed_text = re.sub(r'([\d,.]+) acres?', f"{formatted_lot} acres", processed_text, count=1)
        else:
            # Add to beginning if no mention exists
            processed_text = f"Situated on {formatted_lot} acres. " + processed_text

    return processed_text


def generate_batch_descriptions(
    input_csv_path,
    output_csv_path,
    model_path="./llama3_real_estate_model",
    max_length=150,
    save_interval=5  # Save results every N properties to avoid losing progress
):
    """
    Memory-efficient generator of listing descriptions for properties in a CSV file.

    Args:
        input_csv_path (str): Path to the input CSV file containing property details
        output_csv_path (str): Path to save the output CSV with generated descriptions
        model_path (str): Path to the trained model
        max_length (int): Maximum length of generated descriptions
        save_interval (int): Save progress after processing this many properties
    """
    import pandas as pd
    import torch
    import os
    import gc
    from tqdm import tqdm
    from transformers import AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
    from peft import PeftModel, PeftConfig, get_peft_model

    # Check if CUDA is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device} device")

    # Load the input CSV first
    print(f"Loading data from {input_csv_path}...")
    df = pd.read_csv(input_csv_path)
    print(f"Loaded {len(df)} properties")

    # Add a column for the descriptions if it doesn't exist
    if 'Generated Listing Description' not in df.columns:
        df['Generated Listing Description'] = None

    # Check if we're resuming a previous run
    completed_count = df['Generated Listing Description'].notna().sum()
    if completed_count > 0:
        print(f"Resuming from property {completed_count}/{len(df)}")

    # Load tokenizer first (much lighter on memory)
    print(f"Loading tokenizer from {model_path}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
    except:
        print("Failed to load tokenizer from model path, trying base Llama tokenizer...")
        try:
            # Try to find a PEFT config to get the base model path
            config_path = os.path.join(model_path, "adapter_config.json")
            if os.path.exists(config_path):
                import json
                with open(config_path, 'r') as f:
                    config_data = json.load(f)
                    base_model_path = config_data.get('base_model_name_or_path')
                    print(f"Found base model path: {base_model_path}")
                    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
            else:
                # Fallback to a generic Llama tokenizer
                tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
        except Exception as e:
            print(f"Error loading tokenizer: {e}")
            return None

    def format_property_input(row):
        try:
            # Convert values to appropriate formats with error handling
            try:
                price_in_millions = float(row['Price']) / 1000000
                # We'll keep the price in millions format for the input
            except (ValueError, TypeError):
                price_in_millions = 0.0

            try:
                sq_ft_in_thousands = float(row['Square Footage']) / 1000
                # We'll keep the square footage in thousands format for the input
            except (ValueError, TypeError):
                sq_ft_in_thousands = 0.0

            try:
                lot_size = float(row['Lot Size (Acres)'])
            except (ValueError, TypeError):
                lot_size = 0.0

            # Handle features
            features = row.get('Features', "None")
            if pd.isna(features):
                features = "None"
            elif isinstance(features, list):
                features = ', '.join(features)
            elif isinstance(features, str) and (features.startswith('[') or features.startswith('{')):
                try:
                    evaluated = eval(features)
                    if isinstance(evaluated, (list, dict)):
                        if isinstance(evaluated, dict):
                            features = ', '.join([f"{k}: {v}" for k, v in evaluated.items()])
                        else:
                            features = ', '.join(evaluated)
                except:
                    pass

            # Format the property text - keeping the original format for internal processing
            property_text = (
                f"Price: ${price_in_millions:.6f}M, "
                f"Bedrooms: {row.get('Bedrooms', '0')}, "
                f"Bathrooms: {row.get('Bathrooms', '0')}, "
                f"Square Footage: {sq_ft_in_thousands:.4f}K sq ft, "
                f"Lot Size: {lot_size:.2f} acres, "
                f"Features: {features}"
            )

            return property_text
        except Exception as e:
            print(f"Error formatting property: {e}")
            return f"Price: $0.00M, Bedrooms: 0, Bathrooms: 0, Square Footage: 0.0K sq ft, Lot Size: 0.00 acres, Features: None"

    # Function to load model with minimum memory usage
    def load_model():
        print("Loading model (this may take a while)...")

        # Force garbage collection before loading model
        gc.collect()
        torch.cuda.empty_cache()

        try:
            # First, determine if this is a PEFT model
            is_peft = False
            peft_config = None

            if os.path.exists(os.path.join(model_path, "adapter_config.json")):
                try:
                    peft_config = PeftConfig.from_pretrained(model_path)
                    is_peft = True
                    print(f"Found PEFT config with base model: {peft_config.base_model_name_or_path}")
                except:
                    is_peft = False

            if is_peft and peft_config:
                # Load base model with minimum memory settings
                from transformers import BitsAndBytesConfig

                # 4-bit quantization config
                bnb_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.float16,
                    llm_int8_enable_fp32_cpu_offload=True
                )

                # Load the base model with extreme memory optimization
                base_model = LlamaForCausalLM.from_pretrained(
                    peft_config.base_model_name_or_path,
                    quantization_config=bnb_config,
                    device_map="auto",
                    torch_dtype=torch.float16,
                    low_cpu_mem_usage=True,
                )

                # Load the PEFT adapter
                model = PeftModel.from_pretrained(
                    base_model,
                    model_path,
                    torch_dtype=torch.float16,
                )
            else:
                # Load regular model with minimum memory
                model = LlamaForCausalLM.from_pretrained(
                    model_path,
                    torch_dtype=torch.float16,
                    low_cpu_mem_usage=True,
                    device_map="auto"
                )

            model.eval()
            return model

        except Exception as e:
            print(f"Error loading model: {e}")
            return None

    def generate_listing_description(model, property_info, max_tokens=max_length):
        try:
            # Extract key values
            price_match = re.search(r'Price: \$([\d.]+)M', property_info)
            sqft_match = re.search(r'Square Footage: ([\d.]+)K', property_info)
            lot_match = re.search(r'Lot Size: ([\d.]+) acres', property_info)

            # Convert to desired formats for the prompt
            if price_match:
                price_in_millions = float(price_match.group(1))
                price_in_dollars = int(price_in_millions * 1000000)
                formatted_price = "${:,}".format(price_in_dollars)
            else:
                formatted_price = "unknown price"

            if sqft_match:
                sqft_in_thousands = float(sqft_match.group(1))
                formatted_sqft = str(int(sqft_in_thousands * 1000))
            else:
                formatted_sqft = "unknown square footage"

            if lot_match:
                formatted_lot = lot_match.group(1)
            else:
                formatted_lot = "unknown acreage"

            # Prepare input text with exact values
            input_text = f"<s>[INST] Write a compelling, accurate, and unique real estate listing description for this property. You MUST include and EXACTLY match these numeric values without rounding or changing them: Price: {formatted_price}, Square Footage: {formatted_sqft} sq ft, Lot Size: {formatted_lot} acres. \n\n{property_info} [/INST]"

            # Tokenize
            input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

            # Generate text
            with torch.no_grad():
                output = model.generate(
                    input_ids=input_ids,
                    max_new_tokens=max_tokens,
                    temperature=0.9,
                    top_p=0.95,
                    repetition_penalty=1.2,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id,
                )

            # Decode and extract
            generated_text = tokenizer.decode(output[0], skip_special_tokens=False)

            if "[/INST]" in generated_text:
                description_part = generated_text.split("[/INST]")[1].split("</s>")[0].strip()
            else:
                description_part = generated_text.split("</s>")[0].strip()

            # Apply post-processing to ensure values match exactly
            description_part = post_process_description(description_part, property_info)
            return description_part

        except Exception as e:
            print(f"Error generating description: {e}")
            return "Error generating description"

    # Process properties and generate descriptions
    print("Starting generation process...")

    # Keep track of which rows we've already processed
    processed_indices = df.index[df['Generated Listing Description'].notna()].tolist()
    to_process_indices = [i for i in df.index if i not in processed_indices]

    if len(to_process_indices) > 0:
        # Load model only if we have properties to process
        model = load_model()
        if model is None:
            print("Failed to load model. Exiting.")
            return None

        # Process properties in batches and save periodically
        try:
            for i, idx in enumerate(tqdm(to_process_indices)):
                row = df.iloc[idx]
                property_text = format_property_input(row)
                description = generate_listing_description(model, property_text)
                df.at[idx, 'Generated Listing Description'] = description

                # Save progress periodically
                if (i + 1) % save_interval == 0 or i == len(to_process_indices) - 1:
                    print(f"Saving progress ({i + 1}/{len(to_process_indices)} completed)...")
                    df.to_csv(output_csv_path, index=False)

                # Clean up memory after each generation
                torch.cuda.empty_cache()
                gc.collect()

        except Exception as e:
            print(f"Error during generation: {e}")
        finally:
            # Clean up model to free memory
            del model
            torch.cuda.empty_cache()
            gc.collect()

            # Final save
            print("Saving final results...")
            df.to_csv(output_csv_path, index=False)
    else:
        print("All properties already have descriptions. Nothing to do.")

    print("Process complete!")
    return df


In [3]:
if __name__ == "__main__":
    # Update these paths as needed
    input_csv = "/content/housing_data_simulation test copy.csv"  # Your input CSV file
    output_csv = "housing_data_with_descriptions.csv"  # Where to save the results
    model_dir = "./llama3_real_estate_model"  # Directory containing your trained model

    # Generate descriptions for all properties in the CSV
    generate_batch_descriptions(
        input_csv_path=input_csv,
        output_csv_path=output_csv,
        model_path=model_dir,
        max_length=150,
        save_interval=1  # Save after each property to prevent data loss
    )

Using cpu device
Loading data from /content/housing_data_simulation test copy.csv...
Loaded 50 properties
Loading tokenizer from ./llama3_real_estate_model...
Failed to load tokenizer from model path, trying base Llama tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Error loading tokenizer: Can't load tokenizer for 'meta-llama/Llama-2-7b-hf'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b-hf' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.
